# Input variables:

In [1]:
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import HBox, Label

In [2]:
# set input parameter within givenr ranges
Temp_ = HBox([Label('Temperature [°C]'), widgets.FloatSlider(min=-20.0, max=50.0, value=21.0, step=1)])
Rad_ = HBox([Label('Mean particle radius [$\mu m$]'), widgets.FloatSlider(min=3.22, max=7.22, value=5.22, step=0.01)])
Thick_ = HBox([Label('Electrode thickness [$\mu m$]'), widgets.FloatSlider(min=50.0, max=100.0, value=75.6, step=0.1)])
Rr_coef_ = HBox([Label('Reaction rate coeffic[ent($Am^{-2}(m^3mol^{-1})^{1.5}$]'), widgets.FloatSlider(min=1.5, max=6.5, value=3.42, step=0.01)])
Dif_coef_ = HBox([Label('Diffusion coefficient [$10^{-15} m^2 s^{-1}$]'), widgets.FloatSlider(min=0, max=100, value=1.48, step=0.01)])
Max_c_ = HBox([Label('Maximum lithium concentration [$mol m^{-3}$]'), widgets.FloatSlider(min=0.0, max=100000.0, value=51765.0, step=1)])
SOC_ = HBox([Label('State of charge [%]'), widgets.FloatSlider(min=0, max=100, value=0, step=1)])
Current_ = HBox([Label('Applied current [A]'), widgets.FloatSlider(min=-20.0, max=20.0, value=5.0, step=0.1)])
Vol_per_ = HBox([Label('Active material volume fraction [%]'), widgets.FloatSlider(min = 0.0, max=100.0, value=66.5, step=0.1)])
Area_ = HBox([Label('Electrode plate area [$m^2$]'), widgets.FloatSlider(min=0.001, max=1.0, value=0.1027, step=0.001)])
Sim_steps_ = HBox([Label('No. of simulation steps'), widgets.BoundedIntText(value=1000, min=1, max=10**9, disabled=False)])
Dt_ = HBox([Label('Time step (ns)'), widgets.FloatSlider(min=0.1, max=1000.0, value=2.0, step=0.1)]) 
Out_steps_ = HBox([Label('Output written every [n] steps'), widgets.BoundedIntText(value=5, min=1, max=10**9, disabled=False)])
Space_steps_ = HBox([Label('No. of space steps'), widgets.FloatSlider(min=2, max=1000, value=20, step=1)])
Volt_do_ =  HBox([Label('Output voltage data'), widgets.Select(options=['Yes', 'No'], value='Yes', disabled=False)])
Checkpoint_ =  HBox([Label('Starting from checkpoint file'), widgets.Select(options=['Yes', 'No'], value='No', disabled=False)])

display(Temp_, Rad_, Thick_, Rr_coef_, Dif_coef_, Max_c_, SOC_, Current_, Vol_per_, Area_, Sim_steps_, Dt_, Out_steps_, Space_steps_, Volt_do_, Checkpoint_)

In [3]:
# unbounded boxes to set paramters outside of suggested ranges if wanted
Temp_ = HBox([Label('Temperature [°C]'), widgets.FloatText(value=Temp_.children[1].value, disabled=False)])
Rad_ = HBox([Label('Mean particle radius [$\mu m$]'), widgets.FloatText(value=Rad_.children[1].value, disabled=False)])
Thick_ = HBox([Label('Electrode thickness [$\mu m$]'), widgets.FloatText(value=Thick_.children[1].value, disabled=False)])
Rr_coef_ = HBox([Label('Reaction rate coeffic[ent($Am^{-2}(m^3mol^{-1})^{1.5}$]'), widgets.FloatText(value=Rr_coef_.children[1].value, disabled=False)])
Dif_coef_ = HBox([Label('Diffusion coefficient [$10^{-15} m^2 s^{-1}$]'), widgets.FloatText(value=Dif_coef_.children[1].value, disabled=False)])
Max_c_ = HBox([Label('Maximum lithium concentration [$mol m^{-3}$]'), widgets.FloatText(value=Max_c_.children[1].value, disabled=False)])
Current_ = HBox([Label('Applied current [A]'), widgets.FloatText(value=Current_.children[1].value, disabled=False)])
Area_ = HBox([Label('Electrode area [$m^2$]'), widgets.FloatText(value=Area_.children[1].value, disabled=False)])
Dt_ = HBox([Label('Time step (ns)'), widgets.FloatText(value=Dt_.children[1].value, disabled=False)])
Space_steps_ = HBox([Label('No. of space steps'), widgets.FloatText(value=Space_steps_.children[1].value, disabled=False)])

display(Temp_, Rad_, Thick_, Rr_coef_, Dif_coef_, Max_c_, Current_, Area_, Dt_, Space_steps_)

In [4]:
# saving the values set y the user and adjusting relevant parameters to be in SI units
Temp = Temp_.children[1].value + 273.15   #K
Rad = Rad_.children[1].value * 10**(-6)   #m
Thick = Thick_.children[1].value * 10**(-6)   #m
Rr_coef =  Rr_coef_.children[1].value
Dif_coef = Dif_coef_.children[1].value * 10**(-15)
Max_c = Max_c_.children[1].value
# Init_c derived from values in Chen2020:
# stoichiometry at 0% SOC = 0.2661, and at 100% SOC = 0.9084
# assuming linear behaviour we get x = mcx +c with c = 0.9084
# and m = 0.2661-0.9084 = -0.6423
Init_c = Max_c * ((SOC_.children[1].value/100)*(-0.6423) + 0.9084)
Iapp = Current_.children[1].value / Area_.children[1].value  # A/m^2
Vol_per = Vol_per_.children[1].value
Sim_steps = Sim_steps_.children[1].value
Dt = Dt_.children[1].value * 10**(-9) #m
Out_steps = Out_steps_.children[1].value
Space_steps = Space_steps_.children[1].value
if Volt_do_.children[1].value == 'Yes':
    Volt_do = 1
else:
    Volt_do = 0
if Checkpoint_.children[1].value == 'Yes':
    Checkpoint = 1
else:
    Checkpoint = 0

In [5]:
#create new NetCDF file in 'writing mode' and 'NETCDF4 format'
from netCDF4 import Dataset
rootgrp = Dataset('SPM_input.nc', 'w', format='NETCDF4')
#creating dimensions for vector holding all the input variables
temp_dim = rootgrp.createDimension('temp_dim', 1)
rad_dim = rootgrp.createDimension('rad_dim', 1)
thick_dim = rootgrp.createDimension('thick_dim', 1)
rr_coef_dim = rootgrp.createDimension('rr_coef_dim', 1)
dif_coef_dim = rootgrp.createDimension('dif_coef_dim', 1)
max_c_dim = rootgrp.createDimension('max_c_dim', 1)
init_c_dim = rootgrp.createDimension('init_c_dim', 1)
iapp_dim = rootgrp.createDimension('iapp_dim', 1)
vol_per_dim = rootgrp.createDimension('vol_per_dim', 1)
sim_steps_dim = rootgrp.createDimension('sim_steps_dim', 1)
dt_dim = rootgrp.createDimension('dt_dim', 1)
out_steps_dim = rootgrp.createDimension('out_steps_dim', 1)
space_steps_dim = rootgrp.createDimension('space_steps_dim', 1)
volt_do_dim = rootgrp.createDimension('volt_do_dim', 1)
checkpoint_dim = rootgrp.createDimension('checkpoint_dim', 1)
#createing variable 
temp = rootgrp.createVariable('temp', 'f8', ('temp_dim',))
rad = rootgrp.createVariable('rad', 'f8', ('rad_dim',))
thick = rootgrp.createVariable('thick', 'f8', ('thick_dim',))
rr_coef = rootgrp.createVariable('rr_coef', 'f8', ('rr_coef_dim',))
dif_coef = rootgrp.createVariable('dif_coef', 'f8', ('dif_coef_dim',))
max_c = rootgrp.createVariable('max_c', 'f8', ('max_c_dim',))
init_c = rootgrp.createVariable('init_c', 'f8', ('init_c_dim',))
iapp = rootgrp.createVariable('iapp', 'f8', ('iapp_dim',))
vol_per = rootgrp.createVariable('vol_per', 'f8', ('vol_per_dim',))
sim_steps = rootgrp.createVariable('sim_steps', 'i4', ('sim_steps_dim',))
dt = rootgrp.createVariable('dt', 'f8', ('dt_dim',))
out_steps = rootgrp.createVariable('out_steps', 'i4', ('out_steps_dim',))
space_steps = rootgrp.createVariable('space_steps', 'i4', ('space_steps_dim',))
volt_do = rootgrp.createVariable('volt_do', 'i4', ('volt_do_dim',))
checkpoint = rootgrp.createVariable('checkpoint', 'i4', ('checkpoint_dim',))
# attributes
rootgrp.description = 'Input parameters for SMP model'
temp.description = 'Temperature'
temp.units = 'K'
rad.description = 'Mean particle radius'
rad.units = 'm'
thick.description = 'Electrode thickness'
thick.units = 'm'
rr_coef.description = 'Reaction rate coefficient'
rr_coef.units = '$Am^{-2}(m^3mol^{-1})^{1.5}$'
dif_coef.description ='Diffusion coefficient'
dif_coef.units = '$10^{-15} m^2 s^{-1}$'
max_c.description = 'Maximum lithium concentration'
max_c.units = '$mol m^{-3}$'
init_c.description = 'Initial lithium concentration'
init_c.units = '$mol m^{-3}$'
iapp.description = 'Applied current density'
iapp.units = '$A/m^2$'
vol_per.description = 'Active material volume fraction'
vol_per.units = '%'
sim_steps.description = 'Total number of simultion steps'
sim_steps.units = 'unitless'
dt.description = 'Time step'
dt.units = 's'
out_steps.description = 'Output written every [n] number of steps'
out_steps.units = 'untiless'
space_steps.description = 'No. of space steps'
space_steps.units = 'unitless'
volt_do.description = 'Write voltage data'
volt_do.units = 'unitless'
checkpoint.description = 'Starting from checkpont file'
checkpoint.units = 'unitless'
# writing data to input_parameters variable
temp[0] = Temp
rad[0] = Rad
thick[0] = Thick
rr_coef[0] = Rr_coef
dif_coef[0] = Dif_coef
max_c[0] = Max_c
init_c[0] = Init_c
iapp[0] = Iapp
vol_per[0] = Vol_per
sim_steps[0] = Sim_steps
dt[0] = Dt
out_steps[0] = Out_steps
space_steps[0] = Space_steps
volt_do[0] = Volt_do
checkpoint[0] = Checkpoint
#closing the NetCDF file
rootgrp.close()

In [8]:
print(Iapp)

48.685491723466406
